In [ ]:
import re
import torch
from transformers import MllamaForConditionalGeneration, MllamaProcessor

In [ ]:
model = MllamaForConditionalGeneration.from_pretrained(
    'Bllossom/llama-3.2-Korean-Bllossom-AICA-5B',
    torch_dtype=torch.bfloat16,
    device_map='auto'
)
processor = MllamaProcessor.from_pretrained('Bllossom/llama-3.2-Korean-Bllossom-AICA-5B')

In [3]:
def generate(mode, text):
    if mode == "summary":
        prompt = "다음 텍스트를 요약해줘: "
    elif mode == "quiz":
        prompt = "다음 텍스트로 문제와 정답을 1개만 만들어줘: "
    
    messages = [{'role': 'user',
                 'content': [{'type': 'text',
                              'text': prompt+text}]}]

    input_text = processor.apply_chat_template(messages,
                                               tokenize=False,
                                               add_generation_prompt=True)

    inputs = processor(text=input_text,
                       add_special_tokens=False,
                       return_tensors="pt").to(model.device)

    output = model.generate(**inputs,
                            max_new_tokens=256,
                            temperature=0.1,
                            eos_token_id=processor.tokenizer.convert_tokens_to_ids('<|eot_id|>'),
                            use_cache=False)
    
    output_text = processor.decode(output[0])

    match = re.search(r'<\|start_header_id\|>assistant<\|end_header_id\|>(.*?)<\|eot_id\|>',
                      output_text,
                      re.DOTALL)
    
    return match.group(1)

In [4]:
text = """
엔비디아가 차세대 게이밍 GPU(그래픽처리장치) '지포스 RTX 50시리즈'를 내놓으며 PC용 AI칩 시장을 겨냥한다. 올해 초 미국 라스베이거스서 개최된 CES 2025에서 선보인 바 있는 RTX 50 시리즈는 개인용 AI PC 시장을 바꿔놓을 게임 체인저로 지목되면서 향후 인텔, 퀄컴, AMD 등과의 치열한 각축전이 예상되고 있다. 20일 엔비디아는 서울 코엑스에서 'RTX AI PC 데이'를 열고 새로운 아키텍처 '블랙웰'이 탑재된 '지포스 RTX 50 시리즈'를 발표했다. 이는 RTX는 엔비디아가 만든 GPU(그래픽처리장치)인데 이번에 새롭게 나온 50 시리즈(5090, 5080, 5070TI, 5070)등은 이전 세대보다 강화된 AI(인공지능) 기술과 낮아진 가격을 자랑한다. 이중에서도 5090 제품의 경우 최고급 PC에 주로 사용되며 게이밍, 그래픽 연산 등에 주로 사용된다. 게이머 뿐 아니라 개발자, 크리에이터 등을 위한 최고 사양을 자랑한다. RTX 5090의 경우 초당 데이터 전송량이 초당 1.8TB(테라바이트)로, 이전 모델인 RTX 4090보다 두 배 향상됐다. 성능은 높아졌으나 배터리 효율은 더욱 개선됐다는 것이 엔비디아 측의 설명이다.
"""

In [ ]:
summary = generate("summary", text)
print(summary)

In [ ]:
quiz = generate("quiz", summary)
print(quiz)